In [1]:
import functools
import random

import numpy as np
from cachetools import LFUCache, LRUCache, FIFOCache, RRCache, Cache

from benchmark.beladycache import BeladyCache
from mlfucache import MLFUCache

In [2]:
n = 100000
cbase = n // 100
size = cbase // 20

In [3]:
s = 1.1
HNs = sum([k ** -s for k in range(1, cbase + 1)])


def zipf_pmf(k, s):
    return sum([k ** -s / HNs])


p = 0.01


def geometric_pmf(k, p):
    assert 0 < p <= 1
    return (1 - p) ** (k - 1) * p


keys = range(1, cbase + 1)


In [6]:
def mycached(cache: Cache):
    def decorator(func):
        hits = np.ndarray(shape=(0, 2))

        def wrapper(key):
            nonlocal hits
            try:
                result = cache[key]
                hits = np.append(hits, [[key, True]], axis=0)
                return result
            except KeyError:
                hits = np.append(hits, [[key, False]], axis=0)
            v = func(key)
            try:
                cache[key] = v
            except ValueError:
                pass  # value too large
            return v

        def cache_clear() -> None:
            nonlocal hits
            cache.clear()
            hits = np.ndarray(shape=(0, 2))

        def nrequests() -> int:
            nonlocal hits
            return len(hits)

        def chr() -> float:
            nonlocal hits
            return sum(hits[:, 1]) / len(hits)

        wrapper.cache = cache
        wrapper.cache_clear = cache_clear
        wrapper.nrequests = nrequests
        wrapper.chr = chr
        #        wrapper.cache_info = cache_info

        return functools.update_wrapper(wrapper, func)

    return decorator

In [ ]:
cinfo = {}
for distr, weights in {'zipf': [zipf_pmf(k, s) for k in keys], 'uniform': [1] * cbase,
                       'geometric': [geometric_pmf(k, p) for k in keys]}.items():
    requests = random.choices(keys, weights=weights, k=n)
    for cache in [RRCache(maxsize=size), FIFOCache(maxsize=size), LRUCache(maxsize=size), LFUCache(maxsize=size),
                  MLFUCache(maxsize=size), BeladyCache(maxsize=size, future=requests)]:

        @mycached(cache=cache)
        def storage(key) -> None:
            return None


        assert n == len(requests), f"{len(requests)}"

        for key in requests:
            storage(key)

        assert storage.nrequests() == n, f"requests missmatch {storage.nrequests()}"

        print(f"{distr} - {cache.__class__.__name__}, {storage.chr() * 100:.2f}%")


zipf - RRCache, 50.79%
zipf - FIFOCache, 50.79%
zipf - LRUCache, 56.74%
zipf - LFUCache, 66.81%
zipf - MLFUCache, 68.16%
zipf - BeladyCache, 64.75%
uniform - RRCache, 4.95%
uniform - FIFOCache, 4.95%
uniform - LRUCache, 4.94%
uniform - LFUCache, 5.07%
uniform - MLFUCache, 5.05%
uniform - BeladyCache, 4.93%
geometric - RRCache, 22.97%
geometric - FIFOCache, 22.69%
geometric - LRUCache, 23.72%
geometric - LFUCache, 31.26%
geometric - MLFUCache, 37.70%
